In [ ]:
from datasets import concatenate_datasets

In [ ]:
from datasets import load_dataset

dataset_separated = load_dataset("Thecoder3281f/MIT_separated_final", "normal")
dataset_mixed = load_dataset("Thecoder3281f/MIT_mixed_final", "normal")

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_name = "t5-small"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer.padding_side = "right"
tokenizer.truncation_side = "right"

In [ ]:
def preprocess(batch):
    # inputs = [x.replace(" ", "") for x in batch["input"]]
    # targets = [x.replace(" ", "") for x in batch["target"]]
    inputs = batch["input"]
    targets = batch["target"]

    # print(inputs, targets)

    model_inputs = tokenizer(
        inputs,
        padding="max_length",
        truncation=True,
        max_length=256,
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            padding="max_length",
            truncation=True,
            max_length=256,
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
dataset_separated = dataset_separated.map(preprocess, batched=True, remove_columns=["input", "target"])
dataset_mixed = dataset_mixed.map(preprocess, batched=True, remove_columns=["input", "target"])

In [ ]:
dataset_separated_train = dataset_separated["train"]
dataset_separated_val = dataset_separated["val"]
dataset_separated_test = dataset_separated["test"]

dataset_mixed_train = dataset_mixed["train"]
dataset_mixed_val = dataset_mixed["val"]
dataset_mixed_test = dataset_mixed["test"]

In [ ]:
combined_dataset_train = concatenate_datasets([dataset_separated_train, dataset_mixed_train])
combined_dataset_test = concatenate_datasets([dataset_separated_test, dataset_mixed_test])
combined_dataset_val = concatenate_datasets([dataset_separated_val, dataset_mixed_val])